## Guardrails
Guardrails in LangChain are mechanisms that ensure model responses meet specific technical or formatting requirements.
For example, the JsonValidityEvaluator checks whether the returned text is valid JSON – if the structure is incorrect, the evaluator will return an error.
This allows you to automatically detect and reject responses that are not suitable for further processing.

### JSON Format Validator
Checks if the generated response is valid JSON

In [1]:
from langchain_classic.evaluation import JsonValidityEvaluator

evaluator = JsonValidityEvaluator()
# print(evaluator.evaluate_strings(prediction='{"x": 1}'))      # correct
print(evaluator.evaluate_strings(prediction='{x: 1}'))        # incorrect


{'score': 0, 'reasoning': 'Expecting property name enclosed in double quotes: line 1 column 2 (char 1)'}


### JsonEqualityEvaluator
Checks the equality of JSONs after parsing (the order of keys in JSON does not matter)

In [2]:
from langchain_classic.evaluation import JsonEqualityEvaluator

evaluator = JsonEqualityEvaluator()
print(evaluator.evaluate_strings(
    prediction='{"a":1,"b":[2,3]}',
    reference='{"b":[2,3],"a":2}',
))


{'score': False}


### RegexMatchEvaluator
Checks for a match against a regular expression

In [3]:
from langchain_classic.evaluation import RegexMatchStringEvaluator

evaluator = RegexMatchStringEvaluator()
result = evaluator.evaluate_strings(
    prediction="Order ID: ABC-1234",
    reference=r"^Order ID: [A-Z]{3}-\d{4}$",
)
print(result['score'])

iter = 3
while result['score'] < 1.0 and iter > 0:
    iter -= 1
    print('run model once more')

1


### Fallback Messages Validator
When the main model fails (rate limit / error) automatically use the backup model.

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

primary = ChatOpenAI(model="gpt-4o-miniS", max_retries=0)
backup  = ChatOpenAI(model="gpt-3.5-turbo")

chain = primary.with_fallbacks([backup])

print(chain.invoke("Describe Python in 1 sentence."))


content='Python is a versatile and user-friendly programming language known for its simplicity and readability.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 14, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CZxXq6wdZu8Rwff4AfMJDNDh7ABEn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--7c27ffd5-179c-43a8-ba8d-b5e8407f2529-0' usage_metadata={'input_tokens': 14, 'output_tokens': 16, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### Token Limit
Tracking and pruning history to a token limit to avoid exceeding model context.

In [5]:
import json
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.messages.utils import trim_messages, count_tokens_approximately
from langchain_openai import ChatOpenAI

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="(long conversation history here / many messages...)"),
]

trimmed = trim_messages(
    messages,
    strategy="last",
    token_counter=count_tokens_approximately,
    max_tokens=256,
    start_on="human",
    include_system=True,
)

llm = ChatOpenAI(model="gpt-4o-mini")
print(json.dumps(llm.invoke(trimmed).response_metadata, indent=4))


{
    "token_usage": {
        "completion_tokens": 51,
        "prompt_tokens": 25,
        "total_tokens": 76,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
        },
        "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0
        }
    },
    "model_provider": "openai",
    "model_name": "gpt-4o-mini-2024-07-18",
    "system_fingerprint": "fp_560af6e559",
    "id": "chatcmpl-CZxXrZEP6NDARAHV3gEdZqJBFF6PQ",
    "service_tier": "default",
    "finish_reason": "stop",
    "logprobs": null
}


### Word Limit
Ask for a maximum of N words, count the words after each generation. If this number is exceeded, shorten the answer.

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

limit = 25
prompt = f"Write a summary in MAX {limit} words: What is machine learning?"

resp = llm.invoke(prompt).content
if len(resp.split()) > limit:
    # quick fix - ask the model to shorten to the limit
    resp = llm.invoke(f"Shorten this to max {limit} words, without any additions:\n\n{resp}").content

print(resp)


Machine learning is a subset of artificial intelligence that enables systems to learn and improve from data without explicit programming.
